In [ ]:
import MA4Star_Strategy as MA4
import os
import mplfinance as mpf
import matplotlib.pyplot as plt

In [ ]:
ticker = "QQQ"
window_days = 55
alpha = 0.5
star_lbdays=0
predict_days=5
data_label = MA4.data_label(ticker, window_days, alpha, predict_days, star_lbdays)
print(data_label)

In [ ]:
batch_size = 32
epochs = 30
model_name = "CNN"
# model_name = "Dense"
model_label = MA4.model_label(data_label, model_name, epochs, batch_size)
print(model_label)
model = MA4.retreive_model(model_label)
if model is not None:
    model.summary()

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime

In [ ]:
enddate = date(2025, 7, 3)
enddate = "2025-07-03"
startdate = date(2025, 1, 1)
startdate = "2025-01-01"
print(startdate, ' , ', enddate)

In [ ]:
fulldata = MA4.download_setup_data(ticker=ticker, predict_days=predict_days, window_days=window_days, alpha=alpha, lback=star_lbdays, 
                               start_date=startdate, end_date=enddate)

In [ ]:
validate_charts, validate_labels, finaldata = MA4.generate_chart(data=fulldata, ticker=ticker, predict_days=predict_days, 
                                                             window_days=window_days, datalabel_=data_label)

In [ ]:
def model_predict( model, X_test):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    return y_pred_classes

In [ ]:
def convert2Labels(y_pred_classes):
    label_names=[]
    for pclass in y_pred_classes:
        label_names.append(MA4.LABEL_L[pclass])
    print(len(label_names))
    print(label_names)
    return label_names

In [ ]:
validate_df = pd.DataFrame({"chart_path": validate_charts, "label": validate_labels})
# display(validate_df)
validate_fullp = os.path.join("validate_metadata", f"{model_label}_metadata.csv")
# print(validate_fullp)
os.makedirs(os.path.dirname(validate_fullp), exist_ok=True)
validate_df.to_csv(validate_fullp, index=False)

# display(fulldata.head(30))
df_filtered = finaldata.dropna(subset=['predict_date'])
# display(df_filtered)

validate_X, validate_Y = MA4.load_data(validate_fullp)
print(validate_X.shape, "  ,  ", validate_Y.shape)

y_pred_classes = model_predict(model, validate_X)

y_pred_labels = convert2Labels(y_pred_classes)

df_filtered['pred_Labels'] = y_pred_labels

start_d = df_filtered.index[0].strftime('%Y%m%d')
end_d = df_filtered.index[0].strftime('%Y%m%d')
result_fullp = os.path.join("validate_results", f"{model_label}_{start_d}_{end_d}.csv")
os.makedirs(os.path.dirname(result_fullp), exist_ok=True)
df_filtered.to_csv(result_fullp, index=False)

acc, prec, recall, f1 = MA4.cal_accuracy(ticker, validate_Y, y_pred_classes, model_name, model_label)
print(acc, prec, recall, f1)